In [1]:
import pandas as pd
import requests
import numpy as np
from datetime import datetime, timedelta
import matplotlib

In [2]:
hubs = ['ALBE', 'ALDE', 'AT', 'BE', 'CZ', 'DE_LU', 'FR', 'HR', 'HU', 'NL', 'PL', 'RO', 'SI', 'SK']

start = datetime(2022, 9, 1).strftime('%Y-%m-%d')
end = datetime(2023, 2, 28).strftime('%Y-%m-%d')

In [3]:
def retrieve_net_positions(fromDate, toDate):
    data = {
        'fromUtc': f'{fromDate}T00:00:00.000Z',
        'toUtc': f'{toDate}T23:00:00.000Z',
    }

    try:
        response = requests.post('https://publicationtool.jao.eu/core/api/data/netPos', json=data)
        response.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(err)
        
    return pd.DataFrame(response.json()['data'])

In [4]:
def retrieve_ptdf(fromDate, toDate):
    data = {
        'fromUtc': f'{fromDate}T00:00:00.000Z',
        'toUtc': f'{toDate}T23:00:00.000Z',
    }

    try:
        response = requests.post('https://publicationtool.jao.eu/core/api/data/preFinalComputation', json=data)
        response.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(err)
        
    return pd.DataFrame(response.json()['data'])

In [5]:
def retrieve_atc(fromDate, toDate):
    data = {
        'fromUtc': f'{fromDate}T00:00:00.000Z',
        'toUtc': f'{toDate}T23:00:00.000Z',
    }

    try:
        response = requests.post('https://publicationtool.jao.eu/core/api/data/atc', json=data)
        response.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(err)
        
    return pd.DataFrame(response.json()['data'])

In [7]:
# NET POSITIONS
df_netpos = retrieve_net_positions(start, end)

df_netpos = df_netpos.rename(columns={
    'dateTimeUtc': 'DateTime'
})
df_netpos = df_netpos.set_index('DateTime')
df_netpos.index = pd.to_datetime(df_netpos.index)
df_netpos = df_netpos.tz_localize(None)

df_netpos = df_netpos[[
    'hub_ALBE', 'hub_ALDE', 'hub_AT', 'hub_BE', 'hub_CZ', 'hub_DE', 'hub_FR', 'hub_HR', 'hub_HU', 'hub_NL', 
    'hub_PL', 'hub_RO', 'hub_SI', 'hub_SK'
]]

df_netpos.columns = hubs

df_netpos.to_excel("net_positions.xlsx")

In [9]:
# ZONAL PTDF and RAM
df_ptdf = retrieve_ptdf(start, end)

df_ptdf = df_ptdf.rename(columns={
    'dateTimeUtc': 'DateTime'
})
df_ptdf = df_ptdf.set_index('DateTime')
df_ptdf.index = pd.to_datetime(df_ptdf.index)
df_ptdf = df_ptdf.tz_localize(None)

df_ptdf = df_ptdf[df_ptdf.cneEic != 'NA']
df_ptdf = df_ptdf[[
    'cneEic', 'ram', 'ptdf_ALBE', 'ptdf_ALDE', 'ptdf_AT', 'ptdf_BE', 'ptdf_CZ', 'ptdf_DE', 'ptdf_FR', 'ptdf_HR', 'ptdf_HU', 'ptdf_NL', 
    'ptdf_PL', 'ptdf_RO', 'ptdf_SI', 'ptdf_SK'
]]

df_ptdf.columns = ['eic', 'ram'] + hubs

df_ptdf.to_excel("ptdfs.xlsx")

In [11]:
# NON-FBMC ATC

df_atc = retrieve_atc(start, end)

df_atc = df_atc.rename(columns={
    'dateTimeUtc': 'DateTime'
})
df_atc = df_atc.set_index('DateTime')
df_atc.index = pd.to_datetime(df_atc.index)
df_atc = df_atc.tz_localize(None)
df_atc = df_atc.drop(columns=['id'])

display(df_atc)
df_atc.to_excel("atc.xlsx")

,border_DE_DK1,border_DK1_DE,border_ES_FR,border_FR_ES,border_FR_IT,border_AT_IT,border_IT_AT,border_IT_FR,border_RO_BG,border_BG_RO,border_ALBE_ALDE,border_ALDE_ALBE
DateTime,,,,,,,,,,,,
2022-09-01 00:00:00,2090,2500,2960,3098,NaN,NaN,NaN,NaN,1260,1490,1000,1000
2022-09-01 01:00:00,2100,2500,2960,3098,NaN,NaN,NaN,NaN,1260,1490,1000,1000
2022-09-01 02:00:00,1930,2500,3145,3098,NaN,NaN,NaN,NaN,1260,1490,1000,1000
2022-09-01 03:00:00,2020,2500,3145,3098,NaN,NaN,NaN,NaN,1260,1490,1000,1000
2022-09-01 04:00:00,2120,2500,2543,3098,NaN,NaN,NaN,NaN,1260,1490,1000,1000
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-28 18:00:00,2160,2500,2250,3052,NaN,NaN,NaN,NaN,1814,1509,1000,1000
2023-02-28 19:00:00,2200,2500,2250,3052,NaN,NaN,NaN,NaN,1814,1509,1000,1000
2023-02-28 20:00:00,2230,2500,2250,3052,NaN,NaN,NaN,NaN,1814,1509,1000,1000
